# This is the code which solves the Static Traffic Assignment Problem

In [1]:
import numpy as np

In [68]:
network = 'data/Anaheim'

## 1. We load the graph and the demand
Both graph and demand are in csv file

In [69]:
graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)

In [70]:
# in the case where there is only one o-d, then demand is interpret as a single row and not as a matrix
try:
    demand.shape[1]
except:
    demand = np.array([demand])
nb_ods = int(demand.shape[0])

Then store the links in a dictionary

In [87]:
# graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
graph_dict = {}
for i in range(graph.shape[0]):
    try: 
        graph_dict[int(graph[i][1])]
    except:
        graph_dict[int(graph[i][1])] = {}
    graph_dict[int(graph[i][1])][int(graph[i][2])] = int(graph[i][0])
    
print(graph_dict)

{1: {117: 0}, 2: {87: 1}, 3: {74: 2}, 4: {233: 3}, 5: {165: 4}, 6: {213: 5}, 7: {253: 6}, 8: {411: 7}, 9: {379: 8, 395: 9}, 10: {338: 10, 362: 11}, 11: {309: 12}, 12: {275: 13}, 13: {262: 14}, 14: {257: 15}, 15: {254: 16}, 16: {263: 17}, 17: {276: 18}, 18: {322: 19, 348: 20}, 19: {364: 21, 380: 22}, 20: {397: 23}, 21: {412: 24, 413: 25}, 22: {414: 26, 415: 27}, 23: {416: 28}, 24: {266: 29, 267: 30}, 25: {268: 31, 269: 32}, 26: {273: 33, 274: 34}, 27: {302: 35, 303: 36}, 28: {303: 37, 304: 38}, 29: {308: 39, 337: 40}, 30: {340: 41, 341: 42}, 31: {329: 43, 330: 44}, 32: {332: 45, 333: 46}, 33: {337: 47, 361: 48}, 34: {369: 49, 385: 50}, 35: {373: 51, 389: 52}, 36: {378: 53, 394: 54}, 37: {401: 55, 402: 56}, 38: {406: 57, 407: 58}, 39: {266: 59, 267: 60}, 40: {268: 61, 269: 62}, 41: {273: 63, 274: 64}, 42: {302: 65, 303: 66}, 43: {303: 67, 304: 68}, 44: {308: 69, 337: 70}, 45: {340: 71, 341: 72}, 46: {329: 73, 330: 74}, 47: {332: 75, 333: 76}, 48: {337: 77, 361: 78}, 49: {369: 79, 385: 80

Then, we define the function which gives the travel time as a function of the flow

In [72]:
# TO DO ADD THE CAPACITY
def travel_time(f):
    return graph[:,3] + graph[:,4]*f + graph[:,5]*(f**2) + graph[:,6]*(f**3) + graph[:,7]*(f**4)

In [73]:
nb_links = int(np.max(graph[:,0])+1)
nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
print(nb_nodes)

417


## 2. We compute the all or nothing flow allocation

In [74]:
import scipy

In [75]:
from scipy.sparse.csgraph import dijkstra

To use the Dijkstra's algorithm class of scipy we need to define the adjacent matrix of the graph

In [76]:
def update_travel_time(tt):
    for i in range(graph.shape[0]):
        G[int(graph[i][1])][int(graph[i][2])] = tt[i]

In [90]:
G = np.zeros(shape=(nb_nodes,nb_nodes))
update_travel_time(travel_time(np.zeros(nb_links)))
G = scipy.sparse.csr_matrix(G)
print(G)

  (1, 117)	1.090458488
  (2, 87)	1.090458488
  (3, 74)	1.090458488
  (4, 233)	1.090458488
  (5, 165)	1.090458488
  (6, 213)	1.090458488
  (7, 253)	1.090458488
  (8, 411)	1.0
  (9, 379)	1.0
  (9, 395)	1.0
  (10, 338)	1.0
  (10, 362)	1.0
  (11, 309)	1.0
  (12, 275)	1.0
  (13, 262)	1.0
  (14, 257)	1.0
  (15, 254)	1.0
  (16, 263)	1.0
  (17, 276)	1.0
  (18, 322)	1.0
  (18, 348)	1.0
  (19, 364)	1.0
  (19, 380)	1.0
  (20, 397)	1.0
  (21, 412)	1.0
  :	:
  (407, 53)	1.059848485
  (407, 390)	2.579924242
  (407, 408)	1.079924242
  (407, 416)	2.0
  (408, 211)	0.5
  (408, 407)	1.079924242
  (408, 409)	0.5
  (409, 167)	0.5
  (409, 408)	0.5
  (409, 410)	0.220075758
  (410, 396)	2.0
  (410, 409)	0.220075758
  (410, 411)	2.0
  (411, 8)	1.0
  (411, 410)	2.0
  (412, 21)	1.0
  (412, 402)	2.0
  (413, 21)	1.0
  (413, 404)	2.0
  (414, 22)	1.0
  (414, 405)	2.0
  (415, 22)	1.0
  (415, 406)	2.0
  (416, 23)	1.0
  (416, 407)	2.0


Now let's compute the all or nothing allocation

In [78]:
# computing the all or nothing flow
def all_or_nothing():
    # using scipy to compute dijkstra
    dist_matrix, return_predecessors = dijkstra(G, return_predecessors = True)
    faon = np.zeros(shape = nb_links)
    for i in range(nb_ods):
        o_tmp = int(demand[i][0])
        d_tmp = int(demand[i][1])
        flow_tmp = demand[i][2]

        # Here store the path
        #p = np.zeros()
        node_tmp_d = d_tmp
        while node_tmp_d != o_tmp:
            node_tmp = return_predecessors[o_tmp][node_tmp_d]
            link_tmp = int(graph_dict[node_tmp][node_tmp_d])
            faon[link_tmp] += flow_tmp
            node_tmp_d = node_tmp
    return faon #,p

We define the line search

In [79]:
def potential(graph, f):
    # this routine is useful for doing a line search
    # computes the potential at flow assignment f
    links = int(np.max(graph[:, 0]) + 1)
    g = np.copy(
        graph.dot(np.diag([1., 1., 1., 1., 1 / 2., 1 / 3., 1 / 4., 1 / 5.])))
    x = np.power(f.reshape((links, 1)), np.array([1, 2, 3, 4, 5]))
    return np.sum(np.einsum('ij,ij->i', x, g[:, 3:]))


def line_search(f, res=20):
    # on a grid of 2^res points bw 0 and 1, find global minimum
    # of continuous convex function
    # here we do a bisection
    d = 1. / (2**res - 1)
    l, r = 0, 2**res - 1
    while r - l > 1:
        if f(l * d) <= f(l * d + d):
            return l * d
        if f(r * d - d) >= f(r * d):
            return r * d
        # otherwise f(l) > f(l+d) and f(r-d) < f(r)
        m1, m2 = (l + r) / 2, 1 + (l + r) / 2
        if f(m1 * d) < f(m2 * d):
            r = m1
        if f(m1 * d) > f(m2 * d):
            l = m2
        if f(m1 * d) == f(m2 * d):
            return m1 * d
    return l * d

Now let run the Frank-Wolf's algorithm with a line search to find alpha

In [86]:
eps=1e-8
f = all_or_nothing()
update_travel_time(travel_time(f))

for i in range(1000):
    if i % 100 == 0:
        print(i)
    faon = all_or_nothing() 
    s = line_search(lambda a: potential(graph, (1. - a) * f + a * faon))
    if(i%100==0):
        print(i)
        print(s)
    if s < eps:
        break
    f = (1. - s) * f + s * faon
    update_travel_time(travel_time(f))

0
0
0.12852553386327617
100
100
0.01388865337076695
200
200
0.005190953615155538
300
300
0.0044581741119458255
400
400
0.003383066508288127
500
500
0.0032375156904720396
600
600
0.0025376481961911805
700
700
0.001944053920386326
800
800
0.0012868847728864127
900
900
0.001274989920533545


In [40]:
print(G)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.47875936e+14 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.15000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 2.46559670e+14
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.14000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00

In [41]:
print(f)

[ 2584.94790065     0.          2937.36508258     0.
  2933.31629444    78.82078746  2926.08967403    91.41567024
  2696.46850056   303.46952717  2696.46850056  2584.94790065
 19577.41283935  2837.63926      352.41718193 19125.13862063
   181.02298405    74.77199932 19133.40801979    82.85269335
    84.18904983 19141.93212545    61.61869625    73.8483537
 19457.14726019   104.94134345     0.             0.
     0.          2837.63926       81.16594727  2937.49629678
    87.07330435  2933.27568578    62.91618151  2931.97820052
   190.53530472  2846.38423925  2846.38423925 25000.
 25000.        ]
